In [96]:
import os
import pandas as pd
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.probability import FreqDist
import itertools

In [2]:
os.getcwd()

'C:\\Users\\Vishaal\\Documents\\GitHub\\TREC_Distributed_Machine_Learning\\TREC\\10_Data\\20_Extracted Tweets'

In [18]:
os.chdir("30_2019 B Test")

In [13]:
df1 = pd.read_csv('Ultimate_Dataframe_TREC_2018_train_only_critical.csv')

In [9]:
df2 = pd.read_csv('Ultimate_Dataframe_TREC_2018_test_only_critical.csv')

In [16]:
df3 = pd.read_csv('Ultimate_Dataframe_TREC_2019_A-test_only_critical.csv')

In [19]:
df4 = pd.read_csv('Ultimate_Dataframe_TREC_2019_B-test_only_critical.csv')

In [20]:
df = pd.concat([df1, df2, df3, df4])

In [22]:
df.head()

,Unnamed: 0,ID,Event,Tweet,Categories,Priority
0,87,243376377057902592,costaRicaEarthquake2012,RT @RobDavis_Wx: LANDSLIDE!... road blocked in...,['EmergingThreats'],Critical
1,571,378152455005282306,floodColorado2013,RT @MarciKrivonen: Wow. RT @HemaMullur: Two tr...,['EmergingThreats'],Critical
2,572,378158079579127808,floodColorado2013,RT @HemaMullur: Two trucks and one car in the ...,['EmergingThreats'],Critical
3,619,378220553699463168,floodColorado2013,RT @9NEWS: #COFlood RT @TrevorHughes: Bridge c...,['EmergingThreats'],Critical
4,748,396315108311199744,laAirportShooting2013,Gunshots outside #LAX. Unknown if anyone is in...,['NewSubEvent'],Critical


In [68]:
cat = df['Categories']
cat_l = {}
for element in cat:
    element = element[1:len(element) - 1]
    element = element.split(',')
    for category in element:
        if category in cat_l.keys():
            cat_l[category] = cat_l[category] + 1
        else:
            cat_l[category] = 1
        
        

In [72]:
{k: v for k, v in sorted(cat_l.items(), key=lambda item: item[1], reverse = True)}

{" 'News'": 216,
 " 'Hashtags'": 168,
 " 'Location'": 165,
 " 'EmergingThreats'": 143,
 " 'MultimediaShare'": 136,
 " 'NewSubEvent'": 124,
 "'ThirdPartyObservation'": 100,
 " 'Factoid'": 92,
 " 'Official'": 86,
 "'MovePeople'": 70,
 " 'OriginalEvent'": 69,
 "'Location'": 57,
 "'FirstPartyObservation'": 48,
 "'EmergingThreats'": 40,
 " 'Advice'": 39,
 " 'Discussion'": 20,
 "'Weather'": 18,
 " 'ContextualInformation'": 18,
 " 'Weather'": 17,
 " 'ThirdPartyObservation'": 15,
 "'SearchAndRescue'": 14,
 " 'FirstPartyObservation'": 10,
 " 'ServiceAvailable'": 9,
 " 'Sentiment'": 9,
 "'MultimediaShare'": 9,
 "'Hashtags'": 7,
 "'NewSubEvent'": 6,
 "'InformationWanted'": 6,
 "'Factoid'": 4,
 "'News'": 3,
 "'GoodsServices'": 3,
 "'ServiceAvailable'": 2,
 " 'InformationWanted'": 1,
 "'Official'": 1,
 "'Volunteer'": 1,
 " 'Donations'": 1,
 "'Donations'": 1}

In [80]:
'''
    Function to tokenize, remove stop words and lemmatize
'''
def preProcess(df):
    df['Tweet'] = df['Tweet'].astype('str')
    
    token_array = []
    for tweet in df['Tweet']:
        token_tweet = word_tokenize(tweet)
        token_array.append(token_tweet)
        
    stop_words=set(stopwords.words("english"))
    filtered_token_array=[]
    for tweet in token_array:
        filtered_tweet = []
        for word in tweet:
                if word not in stop_words:
                    filtered_tweet.append(word)
        filtered_token_array.append(filtered_tweet)
        
    lem = WordNetLemmatizer()
    stem = PorterStemmer()

    lemmatized_array=[]
    for tweet in filtered_token_array:
        lemmatized_tweet = []
        for word in tweet:
            lemmatized_tweet.append(lem.lemmatize(word,'v'))
        lemmatized_array.append(lemmatized_tweet)
    
    lemmatized_array_join = []
    for element in lemmatized_array:
        lemmatized_array_join.append(' '.join(element))
        
    return (lemmatized_array_join)


In [84]:
'''
    Get a sense of how big the vocab size is for all the critical tweets
'''
l = preProcess(df)

all_words = []

for tweet in l:
    tokenize_word = word_tokenize(tweet)
    for word in tokenize_word:
        all_words.append(word)

unique_words = set(all_words)
len(unique_words)

2415

In [99]:
'''
    Get the top 50 most frequently used words in the cleaned critical tweets - aka stop words have been removed and 
    tweets have been lemmatised.
'''
df['Tweet'] = df['Tweet'].astype('str')

token_array = []
for tweet in df['Tweet']:
    token_tweet = word_tokenize(tweet)
    token_array.append(token_tweet)

stop_words=set(stopwords.words("english"))
filtered_token_array=[]
for tweet in token_array:
    filtered_tweet = []
    for word in tweet:
            if word not in stop_words:
                filtered_tweet.append(word)
    filtered_token_array.append(filtered_tweet)

lem = WordNetLemmatizer()
lemmatized_array=[]
for tweet in filtered_token_array:
    lemmatized_tweet = []
    for word in tweet:
        lemmatized_tweet.append(lem.lemmatize(word,'v'))
    lemmatized_array.append(lemmatized_tweet)
    
'''
    Joining list of list into a single list
'''
lemmatized_array = list(itertools.chain.from_iterable(lemmatized_array))

'''
    Getting top counts
'''
fdist = FreqDist(lemmatized_array)
top_fifty = fdist.most_common(50)
top_fifty

[(':', 511),
 ('#', 431),
 ('.', 325),
 (',', 261),
 ('https', 239),
 ('@', 163),
 ('RT', 94),
 ('ymmfire', 77),
 ('!', 56),
 ('shoot', 55),
 ('ymm', 55),
 ('fire', 52),
 ('http', 50),
 ('school', 46),
 ('-', 44),
 ('...', 38),
 ('(', 36),
 ('report', 36),
 ('Paris', 36),
 ('say', 35),
 (')', 34),
 ('evacuation', 32),
 ('wildfire', 31),
 ('Colorado', 28),
 (';', 26),
 ('High', 25),
 ('2', 24),
 ('&', 24),
 ('flood', 24),
 ('Alberta', 23),
 ('Level', 23),
 ('BREAKING', 22),
 ("'s", 22),
 ('Ranch', 22),
 ('?', 21),
 ('STEM', 21),
 ('Highlands', 21),
 ('people', 20),
 ('School', 20),
 ('situation', 20),
 ('order', 20),
 ('sheriff', 20),
 ('Lake', 18),
 ('The', 18),
 ('amp', 18),
 ('near', 17),
 ('go', 16),
 ('evacuate', 16),
 ("'", 16),
 ("''", 15)]

In [101]:
'''
    Get the top 50 most frequently used words in the un-cleaned critical tweets - aka stop words have not been removed
'''

df['Tweet'] = df['Tweet'].astype('str')

token_array = []
for tweet in df['Tweet']:
    token_tweet = word_tokenize(tweet)
    token_array.append(token_tweet)
    
'''
    Joining list of list into a single list
'''
token_array = list(itertools.chain.from_iterable(token_array))
fdist_raw = FreqDist(token_array)
top_fifty_raw = fdist.most_common(50)
top_fifty_raw

[(':', 511),
 ('#', 431),
 ('.', 325),
 (',', 261),
 ('https', 239),
 ('@', 163),
 ('RT', 94),
 ('ymmfire', 77),
 ('!', 56),
 ('shoot', 55),
 ('ymm', 55),
 ('fire', 52),
 ('http', 50),
 ('school', 46),
 ('-', 44),
 ('...', 38),
 ('(', 36),
 ('report', 36),
 ('Paris', 36),
 ('say', 35),
 (')', 34),
 ('evacuation', 32),
 ('wildfire', 31),
 ('Colorado', 28),
 (';', 26),
 ('High', 25),
 ('2', 24),
 ('&', 24),
 ('flood', 24),
 ('Alberta', 23),
 ('Level', 23),
 ('BREAKING', 22),
 ("'s", 22),
 ('Ranch', 22),
 ('?', 21),
 ('STEM', 21),
 ('Highlands', 21),
 ('people', 20),
 ('School', 20),
 ('situation', 20),
 ('order', 20),
 ('sheriff', 20),
 ('Lake', 18),
 ('The', 18),
 ('amp', 18),
 ('near', 17),
 ('go', 16),
 ('evacuate', 16),
 ("'", 16),
 ("''", 15)]